# ARI 510 Phase 2b

Project Phase 2b Competition Participation
Ryan Smith
Project Competing in: Yola's team's titled "Bacterial Morphology Classification" which is an image classification task.

At the very bottom of this python notebook there are headings of Full Code for Model 1, and Full Code for Model 2.  The beginning sections here break the code up for explanatory purposes.  You can run the full code from those bottom headings.  There is also multiple full code blocks you can run with different settings listed.

To run these, download the datasets in the GitHub directory that this python notebook is in.  The dataset and file loading is from a local copy, not directly from GitHub.

# Data Loading, Hyperparameters, Preprocessing, and Debug Information

Below we will do a number things, including setting hyperparameters, load the datasets, preprocess the images, and print some debug information.  Part of the hyperparameters are preprocessing, technically, such as scaling and resizing.

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Preprocessing parameters
target_size = (224, 224)  # Resize images to 224x224 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")


Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120


# Check Image Sizes

Below is some debug and information code to check the image sizes.  They vary drastically from about 150x150 to 1000sx1000x.  We resize them to 224x224 for almost all tests, since it performs better than 150x150, with those two values being standards.  

IMPORTANT NOTE: 

Also, this will print 224x224 here, since the images are already resized.  It needs to be inserted before the image resizing.

In [ ]:
# from PIL import Image

def check_image_sizes(directory):
    sizes = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                with Image.open(img_path) as img:
                    sizes.append(img.size)
    return sizes

# Check image sizes in training, validation, and test directories
train_sizes = check_image_sizes(train_dir)
validation_sizes = check_image_sizes(validation_dir)
test_sizes = check_image_sizes(test_dir)

# Print unique sizes
print(f"Unique training image sizes: {set(train_sizes)}")
print(f"Unique validation image sizes: {set(validation_sizes)}")
print(f"Unique test image sizes: {set(test_sizes)}")

Unique training image sizes: {(224, 224)}
Unique validation image sizes: {(224, 224)}
Unique test image sizes: {(224, 224)}


# Data Augmentation

Below is Data Augmentation code that does not test well.  The image shifts and rotations and flips lower the accuracy considerably in my tests.  It is displayed below for reference purposes.

In [ ]:
# preprocessing

# Data augmentation for training and basic scaling for validation/testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotations
    width_shift_range=0.2,  # Horizontal shifts
    height_shift_range=0.2,  # Vertical shifts
    horizontal_flip=True  # Random horizontal flips
)

# Model 1 Building (CNN)

Below we will build our first model which is a CNN model using the Keras implementation Sequential model.

In [ ]:
# Model 1 building

# Model definition
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model 1 Training

Below is the training code:

In [ ]:
# model 1 training

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Model 1 Validation

Below is validation code:

In [ ]:
# Model 1 validation

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Model 1 testing and printing to file

Below is the testing code and printing to file code:

In [ ]:
# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

# Model 2 (Transfer Learning)

Much of the code for the transfer learning model is the same as the previous model's.  I won't go over each individual component for this one.  The full code is below under the heading of 'Full Code for Model 2', along with various tunings of it.

# Full Code for Model 1

Here is various full code views for model 1.

## with data augmentation and image sizes of 224x224:

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import os

# Hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'


# Data augmentation for training and basic scaling for validation/testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotations
    width_shift_range=0.2,  # Horizontal shifts
    height_shift_range=0.2,  # Vertical shifts
    horizontal_flip=True  # Random horizontal flips
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.


Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 485ms/step - accuracy: 0.3195 - loss: 4.4469 - val_accuracy: 0.3229 - val_loss: 1.1246
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2812 - loss: 1.1812 - val_accuracy: 0.3750 - val_loss: 1.1010
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 475ms/step - accuracy: 0.3704 - loss: 1.0954 - val_accuracy: 0.4062 - val_loss: 1.1001
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2812 - loss: 1.1098 - val_accuracy: 0.3333 - val_loss: 1.0759
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 478ms/step - accuracy: 0.3494 - loss: 1.0950 - val_accuracy: 0.3333 - val_loss: 1.0883
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3750 - loss: 1.0917 - val_accuracy: 0.3333 - val_loss: 1.1098
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 475ms/step - accuracy: 0.3999 - loss: 1.0977 - val_accuracy: 0.3125 - val_los

## 150x150 image sizes below:

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Check image sizes
def check_image_sizes(directory):
    sizes = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                with Image.open(img_path) as img:
                    sizes.append(img.size)
    return sizes

# Check image sizes in training, validation, and test directories
train_sizes = check_image_sizes(train_dir)
validation_sizes = check_image_sizes(validation_dir)
test_sizes = check_image_sizes(test_dir)

# Print unique sizes
print(f"Unique training image sizes: {set(train_sizes)}")
print(f"Unique validation image sizes: {set(validation_sizes)}")
print(f"Unique test image sizes: {set(test_sizes)}")

# Data augmentation for training and basic scaling for validation/testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotations
    width_shift_range=0.2,  # Horizontal shifts
    height_shift_range=0.2,  # Vertical shifts
    horizontal_flip=True  # Random horizontal flips
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Unique training image sizes: {(1119, 721), (999, 1000), (700, 499), (1045, 643), (1280, 720), (315, 160), (640, 640), (367, 372), (498, 318), (700, 462), (612, 459), (1024, 931), (300, 300), (841, 538), (419, 400), (201, 251), (513, 691), (1500, 988), (1000, 951), (1300, 1061), (500, 373), (186, 271), (1280, 1003), (1433, 1600), (1083, 800), (426, 378), (1024, 778), (1616, 1216), (1024, 723), (242, 160), (563, 484), (259, 194), (188, 207), (473, 355), (270, 203), (542, 418), (1300, 1289), (1500, 1225), (280, 180), (2079, 2040), (1485, 1600), (366, 213), (500, 238), (1000, 887), (197, 255), (272, 185), (252, 200), (360, 270), (700, 476), (258, 195), (400, 279), (1300, 1008), (474, 301), (1024, 597), (823, 400), (500, 375), (1000, 997), (699, 474), (768, 528), (378, 248), (270, 242), (1563, 1000), (270, 187), (800, 450), (900, 775), (642, 260), (640, 479), (600, 800), (4436, 1740), (360, 254), (1913, 2193), (1000, 999), (650, 465), (1800, 1013), (497, 351), (500, 432), (600, 400), (260, 

## preprocessing but no other data augmentation such as image shifting

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001
target_size = (224, 224)  # Resize images to 224x224 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 470ms/step - accuracy: 0.3353 - loss: 1028.4517 - val_accuracy: 0.3021 - val_loss: 2.5649
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3438 - loss: 2.5275 - val_accuracy: 0.3333 - val_loss: 4.9981
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 457ms/step - accuracy: 0.3778 - loss: 2.1420 - val_accuracy: 0.4062 - val_loss: 1.0631
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4688 - loss: 1.0724 - val_accuracy: 0.2917 - val_loss: 1.1808
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 491ms/step - accuracy: 0.4665 - loss: 1.0068 - val_accuracy: 0.4062 - val_loss: 1.0449
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.8277 - val_accuracy: 0.3333 - val_loss: 1.1338
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 455ms/step - accuracy: 0.5104 - 

## more organized version of the full code, separating hyperparameters and preprocessing

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Preprocessing parameters
target_size = (224, 224)  # Resize images to 224x224 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 481ms/step - accuracy: 0.3684 - loss: 674.5725 - val_accuracy: 0.3646 - val_loss: 6.3188
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3438 - loss: 5.6166 - val_accuracy: 0.3333 - val_loss: 3.9543
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 454ms/step - accuracy: 0.4000 - loss: 2.0983 - val_accuracy: 0.5000 - val_loss: 1.0899
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3750 - loss: 1.0075 - val_accuracy: 0.4583 - val_loss: 1.5674
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 465ms/step - accuracy: 0.4977 - loss: 0.9872 - val_accuracy: 0.4062 - val_loss: 1.2592
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5938 - loss: 0.8683 - val_accuracy: 0.5833 - val_loss: 1.0342
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 483ms/step - accuracy: 0.5491 - l

2024-11-20 23:25:42.053425: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 456ms/step - accuracy: 0.6084 - loss: 0.7712 - val_accuracy: 0.4062 - val_loss: 1.3759
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6250 - loss: 0.6819 - val_accuracy: 0.5000 - val_loss: 2.2282
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.4310 - loss: 1.7876
Validation loss: 1.7037063837051392
Validation accuracy: 0.4791666567325592
Total test images being processed: 120
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Predictions saved to preds.txt


## Hyperparameters of default value

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 1
learning_rate = 0.001

# Preprocessing parameters
target_size = (224, 224)  # Resize images to 224x224 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 501ms/step - accuracy: 0.3889 - loss: 941.4451 - val_accuracy: 0.4062 - val_loss: 2.7928
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.3229 - loss: 2.9035
Validation loss: 2.7656784057617188
Validation accuracy: 0.3645833432674408
Total test images being processed: 120
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step
Predictions saved to preds.txt


## Hyperparameters: Epochs 8, Learning Rate 0.0001, Preprocessing of 256x256 image size

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 8
learning_rate = 0.0001

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(256, 256, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120
Epoch 1/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 700ms/step - accuracy: 0.3294 - loss: 268.3039 - val_accuracy: 0.3958 - val_loss: 39.2884
Epoch 2/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4375 - loss: 70.2922 - val_accuracy: 0.2917 - val_loss: 17.3065
Epoch 3/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 649ms/step - accuracy: 0.3468 - loss: 46.5477 - val_accuracy: 0.3438 - val_loss: 15.9998
Epoch 4/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.2500 - loss: 29.8877 - val_accuracy: 0.3750 - val_loss: 14.1589
Epoch 5/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 596ms/step - accuracy: 0.4270 - loss: 15.6166 - val_accuracy: 0.4688 - val_loss: 2.4501
Epoch 6/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4688 - loss: 3.9952 - val_accuracy: 0.4583 - val_loss: 2.3887
Epoch 7/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 602ms/step - accuracy: 0.3943 

## Hyperparameters of Epochs 12, Learning Rate 0.001, image size of 256x256

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32
epochs = 12
learning_rate = 0.001

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Count total test images
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

print(f"Total test images: {len(test_filenames)}")

# Model definition
model = Sequential([
    Input(shape=(256, 256, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
])

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Total test images: 120
Epoch 1/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 621ms/step - accuracy: 0.3223 - loss: 2460.7139 - val_accuracy: 0.3021 - val_loss: 3.4080
Epoch 2/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3438 - loss: 2.5897 - val_accuracy: 0.5000 - val_loss: 4.0391
Epoch 3/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 670ms/step - accuracy: 0.3855 - loss: 1.6794 - val_accuracy: 0.3854 - val_loss: 1.1415
Epoch 4/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5312 - loss: 1.0409 - val_accuracy: 0.5000 - val_loss: 0.9794
Epoch 5/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 597ms/step - accuracy: 0.4845 - loss: 0.9771 - val_accuracy: 0.4062 - val_loss: 1.0174
Epoch 6/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4688 - loss: 0.9587 - val_accuracy: 0.2917 - val_loss: 1.1020
Epoch 7/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 584ms/step - accuracy: 0.4955 

## keras tuner, similar to gridsearchcv for hyperparameter tuning of keras models.

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image
import keras_tuner as kt

# Hyperparameters
batch_size = 32
epochs = 8

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels
rescale_factor = 1.0 / 255  # Normalize pixel values to [0, 1]

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size, rescale_factor):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size, rescale_factor)
preprocess_images(validation_dir, target_size, rescale_factor)
preprocess_images(test_dir, target_size, rescale_factor)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator()  # No additional preprocessing needed

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Function to create the model
def build_model(hp):
    model = Sequential([
        Input(shape=(256, 256, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')  # Assuming 3 classes: bacilli, cocci, spirilla
    ])
    learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001, 0.00001])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hyperparameter_tuning'
)

# Perform hyperparameter search
tuner.search(train_generator, epochs=epochs, validation_data=validation_generator)

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Evaluate the model on the validation set
val_loss, val_accuracy = best_model.evaluate(validation_generator)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) * rescale_factor  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = best_model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Trial 5 Complete [00h 00m 58s]
val_accuracy: 0.42500001192092896

Best val_accuracy So Far: 0.5666666626930237
Total elapsed time: 00h 04m 50s
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.3690 - loss: 36.8330
Validation loss: 39.39779281616211
Validation accuracy: 0.34166666865348816
Total test images being processed: 120
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
Predictions saved to preds.txt


# Full Code for Model 2

Here are full code views for model 2.

## basic version (MobileNetV2 Transfer Learning)

In [ ]:
# Model 2 full code
# We are using MobileNetV2 as the base model
# This is a Transfer Learning approach
# It is from the Keras module like Sequential was for CNN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32 
epochs = 8
learning_rate = 0.0001

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img)
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size)
preprocess_images(validation_dir, target_size)
preprocess_images(test_dir, target_size)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Use MobileNetV2 preprocessing

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Load the pre-trained MobileNetV2 model from local weights
weights_path = './models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights=None)
base_model.load_weights(weights_path)
base_model.trainable = False  # Freeze the base model

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes: bacilli, cocci, spirilla

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Load test filenames
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Epoch 1/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 306ms/step - accuracy: 0.4017 - loss: 1.4385 - val_accuracy: 0.5521 - val_loss: 0.9222
Epoch 2/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5938 - loss: 0.8897 - val_accuracy: 0.5833 - val_loss: 0.9567
Epoch 3/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 291ms/step - accuracy: 0.5820 - loss: 0.8952 - val_accuracy: 0.6875 - val_loss: 0.7946
Epoch 4/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6250 - loss: 0.8169 - val_accuracy: 0.7083 - val_loss: 0.7330
Epoch 5/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - accuracy: 0.6713 - loss: 0.7304 - val_accuracy: 0.6979 - val_loss: 0.7609
Epoch 6/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5312 - loss: 0.9334 - val_accuracy: 0.7083 - val_loss: 0.6134
Epoch 7/8
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 258ms/step - accuracy: 0.7422 - loss: 0.5635 - val_accuracy: 0.

2024-11-21 12:43:37.812883: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step - accuracy: 0.6940 - loss: 0.7892
Validation loss: 0.7374851107597351
Validation accuracy: 0.7083333134651184
Total test images being processed: 120
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step
Predictions saved to preds.txt


## tuning, modified hyperparameters

See the code comments for the hyperparameters section for the changes.

In [11]:
# Model 2 full code
# We are using MobileNetV2 as the base model
# This is a Transfer Learning approach
# It is from the Keras module like Sequential was for CNN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 16 # from 32 to 16
epochs = 12 # from 8 to 12
learning_rate = 0.001 # from 0.0001 to 0.001

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img)
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size)
preprocess_images(validation_dir, target_size)
preprocess_images(test_dir, target_size)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Use MobileNetV2 preprocessing

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Load the pre-trained MobileNetV2 model from local weights
weights_path = './models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights=None)
base_model.load_weights(weights_path)
base_model.trainable = False  # Freeze the base model

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes: bacilli, cocci, spirilla

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Load test filenames
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Epoch 1/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - accuracy: 0.5565 - loss: 1.1182 - val_accuracy: 0.6786 - val_loss: 0.9726
Epoch 2/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5625 - loss: 1.2289 - val_accuracy: 0.5000 - val_loss: 1.5564
Epoch 3/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.8052 - loss: 0.6497 - val_accuracy: 0.7500 - val_loss: 0.7767
Epoch 4/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9375 - loss: 0.1717 - val_accuracy: 0.7500 - val_loss: 0.3860
Epoch 5/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.8896 - loss: 0.3584 - val_accuracy: 0.7946 - val_loss: 0.7090
Epoch 6/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0762 - val_accuracy: 0.8750 - val_loss: 0.2416
Epoch 7/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.9137 - loss: 0.2035 - val_accuracy

Better results.  From .70 to .78 approximately.

## tuning, default values

In [10]:
# Model 2 full code
# We are using MobileNetV2 as the base model
# This is a Transfer Learning approach
# It is from the Keras module like Sequential was for CNN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 16 # from 32 to 16
epochs = 12 # from 8 to 12
learning_rate = 0.001 # from 0.0001 to 0.001

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img)
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size)
preprocess_images(validation_dir, target_size)
preprocess_images(test_dir, target_size)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Use MobileNetV2 preprocessing

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are already resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Load the pre-trained MobileNetV2 model from local weights
weights_path = './models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights=None)
base_model.load_weights(weights_path)
base_model.trainable = False  # Freeze the base model

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes: bacilli, cocci, spirilla

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Load test filenames
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Epoch 1/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 150ms/step - accuracy: 0.5447 - loss: 1.2104 - val_accuracy: 0.6250 - val_loss: 0.9523
Epoch 2/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7500 - loss: 0.4332 - val_accuracy: 0.2500 - val_loss: 2.9400
Epoch 3/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.8032 - loss: 0.5379 - val_accuracy: 0.7054 - val_loss: 0.8313
Epoch 4/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.1358 - val_accuracy: 0.8750 - val_loss: 0.6350
Epoch 5/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.8723 - loss: 0.3594 - val_accuracy: 0.7054 - val_loss: 0.7647
Epoch 6/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.1183 - val_accuracy: 0.8750 - val_loss: 0.7213
Epoch 7/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.9026 - loss: 0.2488 - val_accuracy

## tuning, image size to 224x224

In [8]:
# Model 2 full code
# We are using MobileNetV2 as the base model
# This is a Transfer Learning approach
# It is from the Keras module like Sequential was for CNN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 16 # from 32 to 16
epochs = 12 # from 8 to 12
learning_rate = 0.001 # from 0.0001 to 0.001

# Preprocessing parameters
target_size = (224, 224)  # Resize images to 224x224 pixels

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img)
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size)
preprocess_images(validation_dir, target_size)
preprocess_images(test_dir, target_size)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Use MobileNetV2 preprocessing

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are now resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are now resized to 224x224 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Load the pre-trained MobileNetV2 model from local weights
weights_path = './models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights=None)
base_model.load_weights(weights_path)
base_model.trainable = False  # Freeze the base model

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes: bacilli, cocci, spirilla

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Load test filenames
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Epoch 1/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - accuracy: 0.5486 - loss: 1.4315 - val_accuracy: 0.7679 - val_loss: 0.7014
Epoch 2/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5000 - loss: 1.3821 - val_accuracy: 0.5000 - val_loss: 0.7047
Epoch 3/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.8327 - loss: 0.4436 - val_accuracy: 0.7411 - val_loss: 0.7047
Epoch 4/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6875 - loss: 0.7733 - val_accuracy: 1.0000 - val_loss: 0.1475
Epoch 5/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - accuracy: 0.8981 - loss: 0.2734 - val_accuracy: 0.7768 - val_loss: 0.6849
Epoch 6/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8125 - loss: 0.3777 - val_accuracy: 0.8750 - val_loss: 0.5545
Epoch 7/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - accuracy: 0.9127 - loss: 0.2132 - val_accuracy

2024-11-21 13:54:59.789374: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.7521 - loss: 0.7639
Validation loss: 0.7883021235466003
Validation accuracy: 0.7410714030265808
Total test images being processed: 120
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step
Predictions saved to preds.txt


## tuning, image size to 150x150

In [7]:
# Model 2 full code
# We are using MobileNetV2 as the base model
# This is a Transfer Learning approach
# It is from the Keras module like Sequential was for CNN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 16 # from 32 to 16
epochs = 12 # from 8 to 12
learning_rate = 0.001 # from 0.0001 to 0.001

# Preprocessing parameters
target_size = (150, 150)  # Resize images to 150x150 pixels

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img)
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size)
preprocess_images(validation_dir, target_size)
preprocess_images(test_dir, target_size)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Use MobileNetV2 preprocessing

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are now resized to 150x150 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are now resized to 150x150 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Load the pre-trained MobileNetV2 model from local weights
weights_path = './models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights=None)
base_model.load_weights(weights_path)
base_model.trainable = False  # Freeze the base model

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes: bacilli, cocci, spirilla

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Load test filenames
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Epoch 1/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.4833 - loss: 1.7878 - val_accuracy: 0.6518 - val_loss: 0.9187
Epoch 2/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7500 - loss: 0.6651 - val_accuracy: 0.6250 - val_loss: 0.8804
Epoch 3/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.8346 - loss: 0.4625 - val_accuracy: 0.7321 - val_loss: 0.8035
Epoch 4/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8750 - loss: 0.4381 - val_accuracy: 0.6250 - val_loss: 1.4350
Epoch 5/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.8125 - loss: 0.4598 - val_accuracy: 0.7054 - val_loss: 0.8293
Epoch 6/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7500 - loss: 0.4987 - val_accuracy: 0.8750 - val_loss: 0.4711
Epoch 7/12
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.9030 - loss: 0.2436 - val_accuracy: 0.

## tuning, image size to 256x256 32/12/.001

In [9]:
# Model 2 full code
# We are using MobileNetV2 as the base model
# This is a Transfer Learning approach
# It is from the Keras module like Sequential was for CNN

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report
import numpy as np
import os
from PIL import Image

# Hyperparameters
batch_size = 32 
epochs = 12 
learning_rate = 0.001

# Preprocessing parameters
target_size = (256, 256)  # Resize images to 256x256 pixels

# Define directories
train_dir = 'yola/split_dataset/train'
validation_dir = 'yola/split_dataset/validation'
test_dir = 'yola/split_dataset/test'
test_filenames_path = 'yola/test_filenames.txt'

# Function to preprocess images
def preprocess_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path)
                img = img.resize(target_size)
                img_array = img_to_array(img)
                img = array_to_img(img_array)
                img.save(img_path)

# Preprocess images in training, validation, and test directories
preprocess_images(train_dir, target_size)
preprocess_images(validation_dir, target_size)
preprocess_images(test_dir, target_size)

# Data generators for training, validation, and test datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Use MobileNetV2 preprocessing

# Load datasets
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # Images are now resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # Images are now resized to 256x256 pixels
    batch_size=batch_size,
    class_mode='categorical'
)

# Debug information
print(f"Total training images: {train_generator.samples}")
print(f"Total validation images: {validation_generator.samples}")

# Load the pre-trained MobileNetV2 model from local weights
weights_path = './models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights=None)
base_model.load_weights(weights_path)
base_model.trainable = False  # Freeze the base model

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes: bacilli, cocci, spirilla

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with the specified learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model on the validation set
validation_steps = validation_generator.samples // batch_size
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")

# Load test filenames
with open(test_filenames_path, 'r') as file:
    test_filenames = file.read().splitlines()

# Predict on test data
test_images = []
for filename in test_filenames:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

test_images = np.vstack(test_images)

print(f"Total test images being processed: {len(test_images)}")

# Predict on test data
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Save predictions to a file
output_file = 'preds.txt'
with open(output_file, 'w') as f:
    for label in predicted_classes:
        f.write(f"{label}\n")

print(f"Predictions saved to {output_file}")

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Total training images: 360
Total validation images: 120
Epoch 1/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 289ms/step - accuracy: 0.4977 - loss: 1.2751 - val_accuracy: 0.7604 - val_loss: 0.6688
Epoch 2/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7188 - loss: 0.7340 - val_accuracy: 0.5833 - val_loss: 0.7965
Epoch 3/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 269ms/step - accuracy: 0.8049 - loss: 0.4888 - val_accuracy: 0.7396 - val_loss: 0.7364
Epoch 4/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7500 - loss: 0.7761 - val_accuracy: 0.5833 - val_loss: 1.1743
Epoch 5/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - accuracy: 0.8779 - loss: 0.3552 - val_accuracy: 0.7083 - val_loss: 0.7917
Epoch 6/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9688 - loss: 0.2413 - val_accuracy: 0.6667 - val_loss: 1.1393
Epoch 7/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 272ms/step - accuracy: 0.9239 - loss: 0.2709 - val_accur